In [ ]:
%matplotlib widget

import keras
import tensorflow as tf
import numpy as np
from sklearn.manifold import TSNE
import numpy as np
from  matplotlib import pyplot as plt

train_ds = tf.keras.utils.image_dataset_from_directory(
        "../data_ssl/test_labeled/",
        seed=5,
        image_size=(224, 224),
        batch_size=32,
        shuffle=True,
        color_mode="rgb",
        label_mode="categorical",
    )

pseudo_labels = np.load("pseudo2_eff.npy")
model = keras.models.load_model("pseudo2_eff.keras")

model2 = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
test_ds = np.concatenate(list(train_ds.take(15).map(lambda x, y : x))) # get five batches of images and convert to numpy array
features = model2(test_ds)
labels = np.argmax(model(test_ds), axis=-1)
tsne = TSNE(n_components=2).fit_transform(features)

def scale_to_01_range(x):

    value_range = (np.max(x) - np.min(x))
    starts_from_zero = x - np.min(x)
    return starts_from_zero / value_range

tx = tsne[:, 0]
ty = tsne[:, 1]

tx = scale_to_01_range(tx)
ty = scale_to_01_range(ty)

colors = ['red', 'blue', 'green', 'brown', 'yellow', "pink", "aqua"]
classes = train_ds.class_names
print(classes)
fig = plt.figure()
ax = fig.add_subplot(111)
for idx, c in enumerate(colors):
    indices = [i for i, l in enumerate(labels) if idx == l]
    current_tx = np.take(tx, indices)
    current_ty = np.take(ty, indices)
    ax.scatter(current_tx, current_ty, c=c, label=classes[idx])

ax.legend(loc='best')
plt.show()